<a href="https://colab.research.google.com/github/kcy919/ml_edu/blob/master/01_%EC%9A%B4%EC%98%81%EC%9D%B4%EB%A0%A5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***PROJECT***: (고양시) 공공자전거 스테이션 최적 위치 선정
  - [고양시_compass](https://https://compas.lh.or.kr/subj/competition/info?subjNo=SBJ_2007_001)
  - [Reference](https://https://www.kaggle.com/kwonyoung234/for-beginner)

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns #시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Mount Google Drive #절대 바꾸지 말 것
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive #
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd drive/'My Drive'/'고양시'

/content/drive/My Drive/고양시


In [ ]:
bicycle_now = pd.read_csv('01.운영이력.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN


## 운영이력 데이터 컬럼
- LEAS_NO    대여 번호
- LEAS_DATE    대여 시간 !
- LEAS_STATION    대여 스테이션 번호 !
- LEAS_DEF_NO    대여 거치대 번호
- RTN_DATE    반납 시간 !
- RTN_STATION    반납 스테이션 번호 !
- RTN_DEF_NO    반납 거치대 번호
- RTN_PROCESS    관제반납구분값 (01:관제반납/02:관리자반납/03:관리자반출/04:타 사용자 반납/05:다른 사용자 반출)
- TRNV_QTY    추정 이동 거리(m) ! -> 평균 이동거리
- MEMB_DIV    회원 구분(비회원은 99이며 나머지는 정회원)
- MEMB_NO    회원 번호
- TEMP_MEMB_NO    비회원 번호
- BIKE_TAG    자전거 번호

In [ ]:
#컬럼['LEAS_DDATE']와 컬럼['RTN_DATE']에서 '년월일'과 '시분초' 쪼개기
# 단, 데이터타입이 'object'여야함
bicycle_now['tempDATE']= bicycle_now.LEAS_DATE.apply(lambda x:x.split())

In [ ]:
bicycle_now['tempDATE_2']= bicycle_now.RTN_DATE.apply(lambda x:x.split())

In [ ]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]"
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]"
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]"
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]"
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]"


In [ ]:
# tempDATE에서 년/월/일 쪼개기
# bicycle_now['LEAS_year']=bicycle_now.tempDATE.apply(lambda x:x[0].split('-')[0])
# bicycle_now['LEAS_month']=bicycle_now.tempDATE.apply(lambda x:x[0].split('-')[1])
# bicycle_now['LEAS_day']=bicycle_now.tempDATE.apply(lambda x:x[0].split('-')[2])

In [ ]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]","[2017-01-01, 00:13:52]",2017,01,01
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]","[2017-01-01, 00:50:24]",2017,01,01
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]","[2017-01-01, 01:01:50]",2017,01,01
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]","[2017-01-01, 00:15:58]",2017,01,01
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]","[2017-01-01, 00:18:44]",2017,01,01


In [ ]:
#tempDATE에서 시/분/초 쪼개기
# bicycle_now['LEAStime']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[0])
# bicycle_now['LEAStime']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[1])
# bicycle_now['LEAStime']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[2])

In [ ]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,LEAStime
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]","[2017-01-01, 00:13:52]",2017,01,01,[00:00:41]
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]","[2017-01-01, 00:50:24]",2017,01,01,[00:01:03]
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]","[2017-01-01, 01:01:50]",2017,01,01,[00:01:50]
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]","[2017-01-01, 00:15:58]",2017,01,01,[00:02:09]
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]","[2017-01-01, 00:18:44]",2017,01,01,[00:03:32]


In [ ]:
# tempDATE에서 시/분/초 쪼개기
# 시/분/초 단위 전처리는 필요 없을 듯함
bicycle_now['LEAS_hour']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[0])
bicycle_now['LEAS_min']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[1])
bicycle_now['LEAS_sec']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[2])

In [ ]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,LEAStime,LEAS_hour,LEAS_min,LEAS_sec
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]","[2017-01-01, 00:13:52]",2017,01,01,[00:00:41],00,00,41
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]","[2017-01-01, 00:50:24]",2017,01,01,[00:01:03],00,01,03
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]","[2017-01-01, 01:01:50]",2017,01,01,[00:01:50],00,01,50
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]","[2017-01-01, 00:15:58]",2017,01,01,[00:02:09],00,02,09
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]","[2017-01-01, 00:18:44]",2017,01,01,[00:03:32],00,03,32


In [ ]:
bicycle_now.tail()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,LEAStime,LEAS_hour,LEAS_min,LEAS_sec
3142683,21718608,2,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,"[2019-12-31, 23:52:39]","[2019-12-31, 23:58:56]",2019,12,31,[23:52:39],23,52,39
3142684,21718610,2,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,"[2019-12-31, 23:53:59]","[2019-12-31, 23:57:59]",2019,12,31,[23:53:59],23,53,59
3142685,21718611,2,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,"[2019-12-31, 23:55:27]","[2020-01-01, 00:06:24]",2019,12,31,[23:55:27],23,55,27
3142686,21718613,2,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,"[2019-12-31, 23:59:14]","[2020-01-01, 00:10:39]",2019,12,31,[23:59:14],23,59,14
3142687,21718614,2,2019-12-31 23:59:30,214,18,2020-01-01 00:10:08,222,3,33169.0,12,215409,0.0,1A84400000F5C7,NaN,"[2019-12-31, 23:59:30]","[2020-01-01, 00:10:08]",2019,12,31,[23:59:30],23,59,30


In [ ]:
bicycle_now.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142688 entries, 0 to 3142687
Data columns (total 22 columns):
 #   Column        Dtype  
---  ------        -----  
 0   LEAS_NO       int64  
 1   LEAS_STAT     int64  
 2   LEAS_DATE     object 
 3   LEAS_STATION  int64  
 4   LEAS_DEF_NO   int64  
 5   RTN_DATE      object 
 6   RTN_STATION   int64  
 7   RTN_DEF_NO    int64  
 8   TRNV_QTY      float64
 9   MEMB_DIV      object 
 10  MEMB_NO       int64  
 11  TEMP_MEMB_NO  float64
 12  BIKE_TAG      object 
 13  RTN_PROCESS   float64
 14  tempDATE      object 
 15  LEAS_year     object 
 16  LEAS_month    object 
 17  LEAS_day      object 
 18  LEAStime      object 
 19  LEAS_hour     object 
 20  LEAS_min      object 
 21  LEAS_sec      object 
dtypes: float64(3), int64(7), object(12)
memory usage: 527.5+ MB


In [ ]:
# tempDATE_2에서 년/월/일 쪼개기
bicycle_now['RTN_year']=bicycle_now.tempDATE_2.apply(lambda x:x[0].split('-')[0])
bicycle_now['RTN_month']=bicycle_now.tempDATE_2.apply(lambda x:x[0].split('-')[1])
bicycle_now['RTN_day']=bicycle_now.tempDATE_2.apply(lambda x:x[0].split('-')[2])

#tempDATE_2에서 시/분/초 쪼개기
bicycle_now['RTNtime']=bicycle_now.tempDATE_2.apply(lambda x:x[1].split(':')[0])
bicycle_now['RTNtime']=bicycle_now.tempDATE_2.apply(lambda x:x[1].split(':')[1])
bicycle_now['RTNtime']=bicycle_now.tempDATE_2.apply(lambda x:x[1].split(':')[2])

In [ ]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,LEAStime,LEAS_hour,LEAS_min,LEAS_sec,RTN_year,RTN_month,RTN_day,RTNtime
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]","[2017-01-01, 00:13:52]",2017,01,01,[00:00:41],00,00,41,2017,01,01,[00:13:52]
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]","[2017-01-01, 00:50:24]",2017,01,01,[00:01:03],00,01,03,2017,01,01,[00:50:24]
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]","[2017-01-01, 01:01:50]",2017,01,01,[00:01:50],00,01,50,2017,01,01,[01:01:50]
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]","[2017-01-01, 00:15:58]",2017,01,01,[00:02:09],00,02,09,2017,01,01,[00:15:58]
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]","[2017-01-01, 00:18:44]",2017,01,01,[00:03:32],00,03,32,2017,01,01,[00:18:44]


In [ ]:
# 대여-반납 시간 구하기
# 데이터유형 변경
#시계열데이터 추출을 위해 'object -> datetime' 으로


bicycle_now["LEAS_DATE"]=pd.to_object(bicycle_now['LEAS_DATE'])
bicycle_now["RTN_DATE"]=pd.to_object(bicycle_now['RTN_DATE'])

NameError: ignored

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

# 분단위로 시간 차 구함
bicycle_now['Diff'] = bicycle_now['RTN_DATE'] - bicycle_now['LEAS_DATE']
bicycle_now['Diff']=bicycle_now['Diff']/np.timedelta64(1,'m')

print(bicycle_now)

          LEAS_NO  LEAS_STAT           LEAS_DATE  ...  RTN_day     RTNtime       Diff
0        15945541          2 2017-01-01 00:00:41  ...       01  [00:13:52]  13.183333
1        15945542          2 2017-01-01 00:01:03  ...       01  [00:50:24]  49.350000
2        15945543          2 2017-01-01 00:01:50  ...       01  [01:01:50]  60.000000
3        15945544          2 2017-01-01 00:02:09  ...       01  [00:15:58]  13.816667
4        15945545          2 2017-01-01 00:03:32  ...       01  [00:18:44]  15.200000
...           ...        ...                 ...  ...      ...         ...        ...
3142683  21718608          2 2019-12-31 23:52:39  ...       31  [23:58:56]   6.283333
3142684  21718610          2 2019-12-31 23:53:59  ...       31  [23:57:59]   4.000000
3142685  21718611          2 2019-12-31 23:55:27  ...       01  [00:06:24]  10.950000
3142686  21718613          2 2019-12-31 23:59:14  ...       01  [00:10:39]  11.416667
3142687  21718614          2 2019-12-31 23:59:30  ... 

- 대여-반납 차를 나타내는 컬럼['Diff']
- 참고 : [Calculate time difference](https://stackoverflow.com/questions/59833174/calculate-time-difference-within-pandas-while-adding-a-new-column)


In [ ]:
# 필요없는 컬럼 삭제
del bicycle_now['LEAS_hour']
del bicycle_now['LEAS_min']
del bicycle_now['LEAS_sec']
del bicycle_now['LEAStime']
del bicycle_now['RTNtime']
del bicycle_now['BIKE_TAG'] #자전거 번호 삭제

In [ ]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,tempDATE_2,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]",2017,01,01,"[2017-01-01, 00:13:52]",2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]",2017,01,01,"[2017-01-01, 00:50:24]",2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]",2017,01,01,"[2017-01-01, 01:01:50]",2017,01,01,60.000000
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]",2017,01,01,"[2017-01-01, 00:15:58]",2017,01,01,13.816667
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]",2017,01,01,"[2017-01-01, 00:18:44]",2017,01,01,15.200000


In [ ]:
#데이터 복구
bicycle = bicycle_now.copy()

In [ ]:
bicycle.head(3)

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,NaN,"[2017-01-01, 00:00:41]","[2017-01-01, 00:13:52]",2017,01,01,2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,NaN,"[2017-01-01, 00:01:03]","[2017-01-01, 00:50:24]",2017,01,01,2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1.0,"[2017-01-01, 00:01:50]","[2017-01-01, 01:01:50]",2017,01,01,2017,01,01,60.000000


In [ ]:
# bicycle_now에서 필요없는 컬럼(거치대 번호) 삭제
del bicycle_now['LEAS_DEF_NO']
del bicycle_now['RTN_DEF_NO']

bicycle_now.head(3)

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,RTN_DATE,RTN_STATION,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,2017-01-01 00:13:52,260,0.0,6,164203,0.0,NaN,"[2017-01-01, 00:00:41]","[2017-01-01, 00:13:52]",2017,01,01,2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,2017-01-01 00:50:24,231,31039.0,1,187551,0.0,NaN,"[2017-01-01, 00:01:03]","[2017-01-01, 00:50:24]",2017,01,01,2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,2017-01-01 01:01:50,0,NaN,12,168994,0.0,1.0,"[2017-01-01, 00:01:50]","[2017-01-01, 01:01:50]",2017,01,01,2017,01,01,60.000000


In [ ]:
# 회원/ 비회원 번호는 무의미. 단 구분은 필요
del bicycle_now['MEMB_NO']
del bicycle_now['TEMP_MEMB_NO']

bicycle_now.head(3)


,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,RTN_DATE,RTN_STATION,TRNV_QTY,MEMB_DIV,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,2017-01-01 00:13:52,260,0.0,6,NaN,"[2017-01-01, 00:00:41]","[2017-01-01, 00:13:52]",2017,01,01,2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,2017-01-01 00:50:24,231,31039.0,1,NaN,"[2017-01-01, 00:01:03]","[2017-01-01, 00:50:24]",2017,01,01,2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,2017-01-01 01:01:50,0,NaN,12,1.0,"[2017-01-01, 00:01:50]","[2017-01-01, 01:01:50]",2017,01,01,2017,01,01,60.000000


In [ ]:
# 이용시간이 가장 긴 노선 기준으로 sorting
bicycle_now.sort_values('Diff', ascending=False)

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,RTN_DATE,RTN_STATION,TRNV_QTY,MEMB_DIV,RTN_PROCESS,tempDATE,tempDATE_2,LEAS_year,LEAS_month,LEAS_day,RTN_year,RTN_month,RTN_day,Diff
1689061,19039828,2,2018-07-18 21:02:35,210,2019-07-19 22:02:03,0,NaN,24,1.0,"[2018-07-18, 21:02:35]","[2019-07-19, 22:02:03]",2018,07,18,2019,07,19,527099.466667
2105654,19795290,2,2018-11-20 21:49:06,310,2018-12-23 17:54:32,211,0.0,7,NaN,"[2018-11-20, 21:49:06]","[2018-12-23, 17:54:32]",2018,11,20,2018,12,23,47285.433333
3096648,21631911,2,2019-12-03 08:33:02,322,2019-12-30 09:33:00,0,NaN,3,1.0,"[2019-12-03, 08:33:02]","[2019-12-30, 09:33:00]",2019,12,03,2019,12,30,38939.966667
1167095,18048753,2,2017-12-09 23:49:16,305,2017-12-26 16:52:47,259,0.0,36,NaN,"[2017-12-09, 23:49:16]","[2017-12-26, 16:52:47]",2017,12,09,2017,12,26,24063.516667
1045026,17811944,2,2017-10-25 09:44:54,234,2017-11-08 10:52:22,236,65535.0,76,NaN,"[2017-10-25, 09:44:54]","[2017-11-08, 10:52:22]",2017,10,25,2017,11,08,20227.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546344,16924455,2,2017-07-01 15:31:39,333,2017-07-01 15:31:39,333,4.0,99,NaN,"[2017-07-01, 15:31:39]","[2017-07-01, 15:31:39]",2017,07,01,2017,07,01,0.000000
2574418,20683572,2,2019-06-24 08:31:20,310,2019-06-24 08:31:20,301,6.0,12,NaN,"[2019-06-24, 08:31:20]","[2019-06-24, 08:31:20]",2019,06,24,2019,06,24,0.000000
2948793,21362666,2,2019-10-13 23:40:01,125,2019-10-13 23:40:01,125,0.0,99,NaN,"[2019-10-13, 23:40:01]","[2019-10-13, 23:40:01]",2019,10,13,2019,10,13,0.000000
2375279,20324010,2,2019-05-01 14:51:41,302,2019-05-01 14:51:41,302,0.0,99,NaN,"[2019-05-01, 14:51:41]","[2019-05-01, 14:51:41]",2019,05,01,2019,05,01,0.000000


['Diff']값이 0 이라는 것은 자전거가 스테이션의 잔류했음을 의미 
-> 전체 자전거 사용량 중 미사용 자전거 추출 용도로 분리

In [ ]:
LEAS_empty=bicycle_now[bicycle_now['Diff']==0.0]['LEAS_STATION']
RTN_empty=bicycle_now[bicycle_now['Diff']==0.0]['RTN_STATION']

empty=pd.DataFrame([LEAS_empty, RTN_empty])
empty

,20,2198,3190,3191,4644,5963,6060,9824,10164,11008,11832,12924,14252,18133,20045,21924,26268,27206,29380,34826,34851,34964,35944,35997,36336,36603,39307,39394,41916,42497,42618,43036,43046,44306,45758,45858,47510,47628,49747,50442,...,3116665,3118972,3119965,3120691,3121990,3121994,3122028,3122058,3122389,3123832,3125584,3126298,3128301,3129574,3129824,3131469,3131661,3131903,3131925,3132241,3133135,3133230,3133619,3135780,3136200,3136770,3137378,3138696,3138725,3138840,3139024,3139454,3139682,3140088,3140589,3140787,3141491,3142052,3142251,3142270
LEAS_STATION,225,323,260,322,306,207,303,220,225,114,302,330,330,218,333,226,330,236,225,330,237,322,317,236,253,321,305,207,309,215,237,232,119,331,148,305,245,254,161,253,...,250,331,330,235,265,265,265,263,304,264,340,127,259,221,231,306,206,306,201,211,317,343,234,129,306,233,265,319,228,231,212,340,253,345,306,235,235,341,301,232
RTN_STATION,225,323,260,322,306,207,303,0,225,0,302,330,330,218,333,226,330,236,225,330,237,322,0,236,253,0,305,207,228,215,237,0,0,331,148,329,226,254,161,253,...,250,331,330,231,265,265,265,263,304,264,340,127,103,221,231,306,206,264,201,265,225,264,234,129,306,233,265,319,228,231,231,340,253,345,306,235,235,0,301,303


결과,

1) ['Diff']= 0 이면서 동시에 ['RTN_STATION']=0 인 값 존재

2) ['LEAS_STATION']=['RTN_STATION'] 스테이션 넘버 추출 가능?

3) ['LEAS_STATION'] != ['RTN_STATION'] 이면서 동시에 ['DIFF']=0 인 데이터도 존재

In [ ]:
# ['Diff'] = 0
# ['LEAS_STATION']=['RTN_STATION'] 인 스테이션 넘버를 추출해서 df으로 만들기


#for문을 사용하지 않아도 된다. => 왜?? 
empty_station = []
for i in range(len(bicycle_now)):
  if bicycle_now['Diff'].iloc[i]==0:
    bicycle_now['LEAS_STATION'].loc[i]=bicycle_now['RTN_STATION'].loc[i]
    print('대여장소 = ', bicycle_now['LEAS_STATION'][i], '반납장소 = ', bicycle['RTN_STATION'][i])
    empty_station.append(bicycle_now['LEAS_STATION'][i])

    
    

empty_station

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
대여장소 =  303 반납장소 =  303
대여장소 =  121 반납장소 =  121
대여장소 =  213 반납장소 =  213
대여장소 =  313 반납장소 =  313
대여장소 =  201 반납장소 =  201
대여장소 =  330 반납장소 =  330
대여장소 =  330 반납장소 =  330
대여장소 =  330 반납장소 =  330
대여장소 =  201 반납장소 =  201
대여장소 =  316 반납장소 =  316
대여장소 =  115 반납장소 =  115
대여장소 =  206 반납장소 =  206
대여장소 =  163 반납장소 =  163
대여장소 =  259 반납장소 =  259
대여장소 =  259 반납장소 =  259
대여장소 =  303 반납장소 =  303
대여장소 =  112 반납장소 =  112
대여장소 =  116 반납장소 =  116
대여장소 =  306 반납장소 =  306
대여장소 =  222 반납장소 =  222
대여장소 =  330 반납장소 =  330
대여장소 =  218 반납장소 =  218
대여장소 =  111 반납장소 =  111
대여장소 =  126 반납장소 =  126
대여장소 =  105 반납장소 =  105
대여장소 =  172 반납장소 =  172
대여장소 =  250 반납장소 =  250
대여장소 =  118 반납장소 =  118
대여장소 =  213 반납장소 =  213
대여장소 =  254 반납장소 =  254
대여장소 =  168 반납장소 =  168
대여장소 =  345 반납장소 =  345
대여장소 =  306 반납장소 =  306
대여장소 =  319 반납장소 =  319
대여장소 =  167 반납장소 =  167
대여장소 =  345 반납장소 =  345
대여장소 =  302 반납장소 =  302
대여장소 =  319 반납장소 =  319
대여장소 =  238 반납장소 =  238
대여장소 =  330 반납장소 =  330
대여장소

[225,
 323,
 260,
 322,
 306,
 207,
 303,
 0,
 225,
 0,
 302,
 330,
 330,
 218,
 333,
 226,
 330,
 236,
 225,
 330,
 237,
 322,
 0,
 236,
 253,
 0,
 305,
 207,
 228,
 215,
 237,
 0,
 0,
 331,
 148,
 329,
 226,
 254,
 161,
 253,
 218,
 0,
 226,
 0,
 161,
 342,
 342,
 343,
 330,
 343,
 0,
 170,
 245,
 0,
 161,
 322,
 213,
 205,
 254,
 339,
 345,
 339,
 101,
 345,
 245,
 0,
 240,
 303,
 328,
 232,
 245,
 222,
 215,
 164,
 168,
 309,
 215,
 238,
 217,
 217,
 161,
 167,
 232,
 303,
 161,
 254,
 0,
 0,
 121,
 161,
 0,
 316,
 218,
 302,
 339,
 124,
 161,
 161,
 234,
 318,
 130,
 233,
 127,
 316,
 306,
 202,
 226,
 232,
 259,
 309,
 101,
 129,
 234,
 345,
 311,
 254,
 322,
 342,
 328,
 311,
 161,
 317,
 320,
 140,
 307,
 304,
 330,
 173,
 253,
 232,
 343,
 321,
 327,
 343,
 227,
 331,
 345,
 112,
 226,
 168,
 345,
 318,
 106,
 219,
 317,
 161,
 116,
 339,
 311,
 314,
 324,
 321,
 0,
 303,
 304,
 233,
 233,
 306,
 231,
 320,
 345,
 225,
 0,
 0,
 317,
 206,
 226,
 342,
 112,
 231,
 331,
 226,
 3

In [ ]:
len(empty_station)

6636

In [ ]:
# empty_station을 DataFrame으로 만들어서 스테이션 중복 카운트
empty_station=pd.DataFrame(empty_station)

In [ ]:
## 중복 요소를 삭제한다 :  drop_duplicates()
empty_station2=empty_station.drop_duplicates()
empty_station2

,0
0,225
1,323
2,260
3,322
4,306
...,...
4307,262
5918,351
6381,265
6516,264


시간차가 없는 자전거들 = 놀고 있는 자전거들 = 비효율성

시간차 없는 자전거들이 위치한 자전거스테이션 : 6636

시간차 없는 자전거들이 위치한 자전거스테이션(중복제거):6592

중복값 : 44개 => 고장날 가능성, 추측 ???? 의미?? 가 있을까 ??? 


In [ ]:
# 시간차가 없는 자전거 스테이션 위치 정보 => 이해 노우 
for i in range(len(empty_station2)):
  if empty_station2[0].iloc[i]==0:
    empty_station2[0].loc[i]=bicycle_station['Station_ID'].loc[i]

    empty_station2.append(bicycle_station['위도'][i])
    empty_station2.append(bicycle_station['경도'][i])


TypeError: ignored

In [ ]:
empty = 

In [ ]:
bicycle_now['LEAS_STATION']= pd.to_datetime(bicycle_now['RTN_DATE'])

##대여 반납 시간대 조사 를 위한 시간대 별로 쪼개기 카운트 

- 최장기 이용 기록 : [2018.07.18-2019.07.19] 1년
- 최단기 이용 기록 : 스테이션 체류

  + 예측1) 시간차[Diff]가 0 아이들을 제외하고 연/월을 기준으로 많은 사용 기간대 추출
  + 예측2) 가장 많은 대여(LEAS) / 반납(RTN) 번호 세기


In [ ]:
# 이용 시간에 따른 순서 배열 

- 위도 경도값을 통한 거리 구하기
https://stricky.tistory.com/283
https://mizykk.tistory.com/21

In [ ]:
# 대여/반납 스테이션 df 분리
# 위도/경도 값 입혀서 harsine 함수 입혀서 거리 추출
LEAS_distance = bicycle_now['LEAS_STATION']

RTN_distance = bicycle_now['RTN_STATION']

- 01.운영이력 파일에는 위도/경도가 없고, 사용 거리 데이터가 누락치가  많음.


- 대여/반납 df에 위도/경도 입히기

In [ ]:
# 런타임 연결 해결 
# 대여/반납 df에 각각 위도 경도 입히기
bicycle_station = pd.read_csv('02.자전거스테이션.csv')

In [ ]:
bicycle_station.head(100)

,Station_ID,STATION_NAME,거치대 수량,위도,경도
0,101,어울림마을 701동 앞,20,37.654775,126.834584
1,103,대림e-편한세상106동,20,37.660442,126.840377
2,104,탄현마을8단지,25,37.698523,126.766042
3,105,KT 덕양지사 앞,20,37.655244,126.839261
4,106,원당역 앞 공영주차장,30,37.653410,126.842530
...,...,...,...,...,...
95,242,◆식사 3단지 분수대 옆,43,37.681718,126.812546
96,245,★하늘마을4단지 403동 앞,20,37.675567,126.784853
97,246,하늘마을1단지 101동 앞,20,37.680743,126.781692
98,247,안곡중학교 옆,25,37.680703,126.785326


In [ ]:
# 대여 - 위도/경도


In [ ]:
#반납 - 위도/경도

In [ ]:
# 위도/경도를 통한 거리 